# PROYECTO WEB SACRAPING


El objetivo de este proyecto es realizar webscraping de las publicaciones de mercadolibre sobre un determinado modelo de productos "enova" para hacer una clasificacion segun los precios, modelo, spec. Luego de este analisis podremos visivilizar de forma rapida y eficiente los precios mas economicos que hay en el mercado comparado con los de la marca "enova".

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import etree
from datetime import date
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import re
import tkinter as tk
from tkinter import ttk
from tkinter import simpledialog
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import os
from tqdm import tqdm
import threading
from concurrent.futures import ThreadPoolExecutor

In [2]:
def obtener_url():
    root = tk.Tk()
    root.title("Ingreso de URL")
    
    style = ttk.Style()
    style.theme_use('clam')  

    window_width = 400
    window_height = 200
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    position_top = int(screen_height / 2 - window_height / 2)
    position_right = int(screen_width / 2 - window_width / 2)
    root.geometry(f'{window_width}x{window_height}+{position_right}+{position_top}')
    
    
    root.attributes('-topmost', 1)

   
    ttk.Label(root, text="Por favor, ingrese la URL que desea analizar:").pack(pady=20)
    url_entry = ttk.Entry(root, width=50)
    url_entry.pack(pady=10)

    def on_submit():
        global url_input
        url_input = url_entry.get()
        root.destroy()

    ttk.Button(root, text="Enviar", command=on_submit).pack(pady=20)

    root.mainloop()
    return url_input


  

In [3]:
url_input = obtener_url()
response = requests.get(url_input)


In [4]:
response.content

b''

In [5]:
if response.status_code != 200:
    print("Error al obtener la página. Código de estado:", response.status_code)
else:
    soup = BeautifulSoup(response.content, 'html.parser')
    
response.status_code


Error al obtener la página. Código de estado: 403


403

In [6]:
titulos = soup.find_all('h2', class_='ui-search-item__title')
titulos[0].text


NameError: name 'soup' is not defined

In [ ]:
for i in titulos:
    
    print(i.text)

In [ ]:
titulos = [i.text for i in titulos]
titulos

In [ ]:
len(titulos)

In [ ]:
urls = soup.find_all('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

In [ ]:
urls[0].get('href')

In [ ]:
urls = [i.get('href') for i in urls]
urls

In [ ]:
print(len(titulos))
len(urls)


In [ ]:
df = pd.DataFrame({'Titulos':titulos, 'URL':urls})
df['Fecha'] = date.today()
df

In [ ]:
precio = soup.find_all('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript')
precio

In [ ]:
precio = [p.text for p in precio]
precio

In [ ]:
len(precio)

In [ ]:
df = pd.DataFrame({'Titulos':titulos, 'URL':urls, 'Precio':precio })
df['Fecha'] = date.today()
df

In [ ]:
def limpiar_precio(precio):
    
    if 'US$' in precio:
        
        precio_limpio = re.sub(r'[^\d,]', '', precio.replace('US$', ''))
        precio_limpio = precio_limpio.replace(',', '')
        return f"${float(precio_limpio) * 1300:,.0f}"
    else:
        
        precio_limpio = re.sub(r'[^\d,]', '', precio)
        precio_limpio = precio_limpio.replace(',', '')
        return f"${float(precio_limpio):,.0f}"


df['Precio'] = df['Precio'].apply(limpiar_precio)
df['Precio']

In [ ]:
def extraer_stock_disponible(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            stock_info = soup.find('span', class_='ui-pdp-buybox__quantity__available')
            if stock_info:
                stock_text = stock_info.get_text()
                stock_number = re.search(r'\d+', stock_text)
                if stock_number:
                    return int(stock_number.group())
    except requests.RequestException as e:
        print(f"Error al acceder a {url}: {e}")
    return 0  

def realizar_extraccion(df):

    with ThreadPoolExecutor(max_workers=10) as executor:
        stocks = list(executor.map(extraer_stock_disponible, df['URL']))

    df['Stock Disponible'] = stocks
    df['Stock Disponible'] = df['Stock Disponible'].apply(lambda x: 'Última Disponible' if x == 0 else x)
    
    return df

df = realizar_extraccion(df)
print(df)


In [ ]:
def precio_numerico(precio):
    precio_limpio = re.sub(r'[^\d]', '', precio)
    return float(precio_limpio)

df['precio_numerico'] = df['Precio'].apply(precio_numerico)

df_sorted = df.sort_values(by='precio_numerico', ascending=True)

df_sorted.columns = [str(col).strip() for col in df_sorted.columns]

df_sorted

In [ ]:
def graficar_precios():
    producto_enova = df_sorted[df_sorted['Titulos'].str.contains('Enova', case=False, na=False)]
    
    if not producto_enova.empty:
        precio_enova = producto_enova.iloc[0]['precio_numerico']
        productos_mas_baratos = df_sorted[df_sorted['precio_numerico'] < precio_enova]

        fig, ax = plt.subplots(figsize=(10, 9))
        sns.barplot(x=productos_mas_baratos['Titulos'], y=productos_mas_baratos['precio_numerico'], palette='viridis', ax=ax)
        ax.set_xlabel('Productos')
        ax.set_ylabel('Precio (en pesos)')
        ax.set_title('Precios más baratos que el producto "Enova"')
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
        plt.tight_layout()
        
        grafico_ventana = tk.Toplevel()
        grafico_ventana.title("Gráfico de precios")
       
        canvas = FigureCanvasTkAgg(fig, master=grafico_ventana)
        canvas.draw()
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        
        button = tk.Button(master=grafico_ventana, text="Cerrar", command=grafico_ventana.destroy)
        button.pack(side=tk.BOTTOM)

        grafico_ventana.mainloop()
    else:
        print("No se encontró ningún producto con 'Enova' en el título.")

def mostrar_dataframe(df_sorted):
  
    ventana_secundaria = tk.Toplevel()
    ventana_secundaria.title("Listado de Productos Ordenados por precio")
    
    frame = tk.Frame(ventana_secundaria)
    frame.pack(fill=tk.BOTH, expand=1)

   
    columns = [str(col) for col in df_sorted.columns]
    tree = ttk.Treeview(frame, columns=columns, show='headings')
    tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=1)

    
    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, anchor='center')
    
    for index, row in df_sorted.iterrows():
        tags = ('enova',) if 'Enova' in row['Titulos'] else ('normal',)
        tree.insert("", tk.END, values=list(row), tags=tags)
  
    scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=tree.yview)
    tree.configure(yscroll=scrollbar.set)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
    
    tree.tag_configure('enova', background='lightgreen')
    tree.tag_configure('normal', background='white')

    siguiente_button = tk.Button(ventana_secundaria, text="Graficar", command=graficar_precios)
    siguiente_button.pack(pady=10)


root = tk.Tk()
root.withdraw() 

def obtener_nombre_archivo_consecutivo(base_name, extension):
    
    files = os.listdir('.')
    
    pattern = re.compile(rf'{re.escape(base_name)}(\d*)\.{re.escape(extension)}')
    matching_files = [f for f in files if pattern.match(f)]
    
    numbers = []
    for file in matching_files:
        match = pattern.match(file)
        if match.group(1):
            numbers.append(int(match.group(1)))
        else:
            numbers.append(0)
    
    if numbers:
        next_number = max(numbers) + 1
    else:
        next_number = 1
    
    new_filename = f"{base_name}{next_number}.{extension}"
    return new_filename


base_name = 'productos_ordenados'
extension = 'csv'
csv_filename = obtener_nombre_archivo_consecutivo(base_name, extension)
df_sorted.to_csv(csv_filename, index=False)
print(f"DataFrame guardado en {csv_filename}")
os.startfile(csv_filename)

mostrar_dataframe(df_sorted)

root.mainloop()
